In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn
from topic_model import TopicModel

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '4', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/tglm_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 500, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 30, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

## feed dict

In [8]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow], name='bow')
t_variables['input_token_idxs'] = tf.placeholder(tf.int32, [None, None], name='input_token_idxs')
t_variables['dec_input_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_input_idxs')
t_variables['dec_target_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_target_idxs')
t_variables['batch_l'] = tf.placeholder(tf.int32, name='batch_l')
t_variables['doc_l'] = tf.placeholder(tf.int32, [None], name='doc_l')
t_variables['sent_l'] = tf.placeholder(tf.int32, [None], name='sent_l')
t_variables['keep_prob'] = tf.placeholder(tf.float32, name='keep_prob')

In [9]:
def get_feed_dict(batch, mode='train', assertion=False):
    def token_dropout(sent_idxs):
        sent_idxs_dropout = np.asarray(sent_idxs)
        sent_idxs_dropout[np.random.rand(len(sent_idxs)) > config.word_keep_prob] = config.UNK_IDX
        return list(sent_idxs_dropout)

    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    
    doc_l = np.array([len(instance.token_idxs) for instance in batch])
    
    feed_input_token_idxs_list = [sent_idxs for instance in batch for sent_idxs in instance.token_idxs]
    feed_dec_input_idxs_list = [[config.BOS_IDX] + token_dropout(sent_idxs) for sent_idxs in feed_input_token_idxs_list]
    feed_dec_target_idxs_list = [sent_idxs + [config.EOS_IDX]  for sent_idxs in feed_input_token_idxs_list]
        
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    batch_l = len(sent_l)
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_input_idxs = pad_sequences(feed_dec_input_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_target_idxs = pad_sequences(feed_dec_target_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    
    if assertion:
        index = 0
        for instance in batch:
            for line_idxs in instance.token_idxs:
                assert feed_input_token_idxs_list[index] == line_idxs
                index += 1
        assert feed_input_token_idxs.shape[1] == np.max(sent_l)
        assert feed_dec_input_idxs.shape[1] == np.max(sent_l) + 1
        assert feed_dec_target_idxs.shape[1] == np.max(sent_l) + 1
    
    keep_prob = config.keep_prob if mode == 'train' else 1.0

    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['batch_l']: batch_l, t_variables['doc_l']: doc_l, t_variables['sent_l']: sent_l, 
                t_variables['input_token_idxs']: feed_input_token_idxs, t_variables['dec_input_idxs']: feed_dec_input_idxs, t_variables['dec_target_idxs']: feed_dec_target_idxs, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    
# sent_loss_kl_categ_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, tf_log(prob_topic_infer/prob_topic_sents)), 1))
# debug_value([sent_loss_kl_categ, sent_loss_kl_categ_tmp])
# sent_loss_kl_gauss_tmp = 0.5 * tf.reduce_sum(tf.exp(logvars_topic_infer-logvars_topic) + tf.square(means_topic - means_topic_infer) / tf.exp(logvars_topic) - 1 + (logvars_topic - logvars_topic_infer), -1)
# sent_loss_kl_gmm_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss_tmp), -1))
# debug_value([sent_loss_kl_gmm_tmp, sent_loss_kl_gmm])    

## topic model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

    # prior of each gaussian distribution (computed for each topic)
    hidden_topic = tf.layers.Dense(units=config.dim_hidden_topic, activation=tf.nn.relu, name='hidden_topic')(topic_bow)
    means_topic = tf.layers.Dense(units=config.dim_latent, name='mean_topic')(hidden_topic)
    logvars_topic = tf.layers.Dense(units=config.dim_latent, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_topic)
    sigma_topic = tf.exp(0.5 * logvars_topic)
    gauss_topic = tfd.Normal(loc=means_topic, scale=sigma_topic)    
    
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))
# topic_angles = tf.acos(topic_dots)
# topic_angles_mean = tf.reduce_mean(topic_angles)
# topic_angles_vars = tf.reduce_mean(tf.square(topic_angles - topic_angles_mean))
# topic_loss_reg = tf.exp(topic_angles_vars - topic_angles_mean)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

## encoder

In [12]:
# input
input_token_idxs = t_variables['input_token_idxs']
batch_l = t_variables['batch_l']
sent_l = t_variables['sent_l']
max_sent_l = tf.reduce_max(sent_l)

with tf.variable_scope('sent/enc', reuse=False):
    # get word embedding
    enc_input = tf.nn.embedding_lookup(embeddings, input_token_idxs)

    # get sentence embedding
    _, enc_state = dynamic_bi_rnn(enc_input, sent_l, config.dim_hidden, t_variables['keep_prob'])

    # TODO House Holder flow
    hidden_topic_infer =  tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='hidden_topic_infer')(enc_state)
    prob_topic_infer = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic_infer')(hidden_topic_infer)

    w_mean_topic_infer = tf.get_variable('mean_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32)
    b_mean_topic_infer = tf.get_variable('mean_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32)
    means_topic_infer = tf.tensordot(enc_state, w_mean_topic_infer, axes=[[1], [1]]) + b_mean_topic_infer
    
    w_logvar_topic_infer = tf.get_variable('logvar_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    b_logvar_topic_infer = tf.get_variable('logvar_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    logvars_topic_infer = tf.tensordot(enc_state, w_logvar_topic_infer, axes=[[1], [1]]) + b_logvar_topic_infer
    sigma_topic_infer = tf.exp(0.5 * logvars_topic_infer)
    gauss_topic_infer = tfd.Normal(loc=means_topic_infer, scale=sigma_topic_infer)
    
    # latent vectors from each gaussian dist.
    latents_topic_infer = sample_latents(means_topic_infer, logvars_topic_infer) 
    # latent vector from gaussian mixture
    latents_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), latents_topic_infer, transpose_a=True)
    
    # for beam search
    means_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), means_topic_infer, transpose_a=True)    

## decoder

In [13]:
# prepare for decoding
dec_sent_l = tf.add(sent_l, 1)
dec_input_idxs = t_variables['dec_input_idxs']
dec_input = tf.nn.embedding_lookup(embeddings, dec_input_idxs)

dec_latents_input = tf.tile(latents_input, [1, tf.shape(dec_input)[1], 1])
dec_concat_input = tf.concat([dec_input, dec_latents_input], -1)

# decode for training
with tf.variable_scope('sent/dec/rnn', initializer=tf.contrib.layers.xavier_initializer(), dtype = tf.float32, reuse=False):
    dec_cell = tf.contrib.rnn.GRUCell(config.dim_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = t_variables['keep_prob'])

    dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(latents_input, 1))
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_concat_input, sequence_length=dec_sent_l)

    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=helper,
        initial_state=dec_initial_state)

    dec_outputs, _, output_sent_l = tf.contrib.seq2seq.dynamic_decode(train_decoder)
    
    output_layer = tf.layers.Dense(config.n_vocab, use_bias=False, name='out')
    output_logits = output_layer(dec_outputs.rnn_output)
    
    output_token_idxs = tf.argmax(output_logits, 2)

In [14]:
start_tokens = tf.fill([batch_l], config.BOS_IDX)
end_token = config.EOS_IDX

with tf.variable_scope('sent/dec/rnn', reuse=True):
    infer_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(means_input, 1))
    beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(infer_dec_initial_state, multiplier=config.beam_width)
    beam_latents_input = tf.contrib.seq2seq.tile_batch(tf.squeeze(means_input, 1), multiplier=config.beam_width) # added
    
    beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=beam_latents_input)

    beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        beam_decoder,
        maximum_iterations = config.maximum_iterations)

    beam_output_token_idxs = beam_dec_outputs.predicted_ids[:, :, 0]

In [15]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    inter_means_input = tf.placeholder(tf.float32, [None, config.dim_latent])
    
    inter_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(inter_means_input)
    inter_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(inter_dec_initial_state, multiplier=config.beam_width)
    inter_beam_latents_input = tf.contrib.seq2seq.tile_batch(inter_means_input, multiplier=config.beam_width) # added
    
    inter_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=inter_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=inter_beam_latents_input)

    inter_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inter_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    inter_beam_output_token_idxs = inter_beam_dec_outputs.predicted_ids[:, :, 0]

In [16]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    topic_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic)
    topic_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(topic_dec_initial_state, multiplier=config.beam_width)
    topic_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic, multiplier=config.beam_width) # added
    
    topic_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=topic_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=topic_beam_latents_input)

    topic_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        topic_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    topic_beam_output_token_idxs = topic_beam_dec_outputs.predicted_ids[:, :, 0]

In [17]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    means_topic_summary = tf.reduce_mean(means_topic_infer, 0)
    
    summary_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic_summary)
    summary_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(summary_dec_initial_state, multiplier=config.beam_width)
    summary_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic_summary, multiplier=config.beam_width) # added
    
    summary_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=summary_beam_dec_initial_state,
        beam_width=config.beam_width,
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=summary_beam_latents_input)

    summary_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        summary_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    summary_beam_output_token_idxs = summary_beam_dec_outputs.predicted_ids[:, :, 0]

## language modeling cost

In [18]:
# target and mask
dec_target_idxs = t_variables['dec_target_idxs']
dec_mask_tokens = tf.sequence_mask(dec_sent_l, maxlen=max_sent_l+1, dtype=tf.float32)

# nll for each token (averaged over batch & sentence)
sent_loss_recon = tf.contrib.seq2seq.sequence_loss(output_logits, dec_target_idxs, dec_mask_tokens)

In [19]:
doc_l = t_variables['doc_l']
mask_sents = tf.sequence_mask(doc_l)
mask_sents_flatten = tf.reshape(mask_sents, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1]])

prob_topic_tiled = tf.tile(tf.expand_dims(prob_topic, 1), [1, tf.shape(mask_sents)[1], 1])
prob_topic_flatten = tf.reshape(prob_topic_tiled, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1], config.n_topic])
prob_topic_sents = tf.boolean_mask(prob_topic_flatten, mask_sents_flatten)

In [20]:
# inferred mixture probabilities (computed for each sentence)
categ_topic_infer = tfd.Categorical(probs=prob_topic_infer)

# prior of mixture probabilities (computed for each document, tiled for each sentence)
categ_topic = tfd.Categorical(probs=prob_topic_sents)

sent_loss_kl_categ = tf.reduce_mean(tfd.kl_divergence(categ_topic_infer, categ_topic))

# inference of each gaussian gaussribution (computed for each sentence)

sent_loss_kl_gauss = tf.reduce_sum(tfd.kl_divergence(gauss_topic_infer, gauss_topic), -1)
sent_loss_kl_gmm = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss), -1))

sent_loss_kl = sent_loss_kl_categ + sent_loss_kl_gmm

## optimizer

In [21]:
global_step = tf.Variable(0, name='global_step',trainable=False)
tau = tf.cast(tf.divide(tf.mod(global_step, tf.constant(config.cycle_steps)), tf.constant(config.cycle_steps)), dtype=tf.float32)
beta = tf.minimum(1., tau/config.r_cycle)

sent_loss = sent_loss_recon + beta * sent_loss_kl

topic_loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg
loss = topic_loss + sent_loss

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)
    
grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]

opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# run model 

In [22]:
def idxs_to_sents(token_idxs, config, idx_to_word):
    sents = []
    for sent_idxs in token_idxs:
        tokens = []
        for idx in sent_idxs:
            if idx == config.EOS_IDX: break
            tokens.append(idx_to_word[idx])
        sent = ' '.join(tokens)
        sents.append(sent)
    return sents

In [23]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean, ppl_mean

def get_all_losses(sess, batches):
    losses = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch = \
        sess.run([loss, topic_loss_recon, topic_loss_kl, sent_loss_recon, sent_loss_kl], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
    print('LOSS %.2f | TM NLL: %.2f, KL: %.4f | LM NLL: %.2f, KL: %.4f' %  np.mean(losses, 0))

In [24]:
def print_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch)
    pred_token_idxs = sess.run(output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for i, (true_sent, pred_sent) in enumerate(zip(true_sents, pred_sents)):        
        print(i, 'TRUE: %s' % true_sent)
        print(i, 'PRED: %s' % pred_sent)

def print_topic_sample():
    pred_topics_freq_bow_indices, pred_topic_token_idxs = sess.run([topics_freq_bow_indices, topic_beam_output_token_idxs], 
                                                                                                           feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})
    pred_topic_sents = idxs_to_sents(pred_topic_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Topic Samples-----------')
    for i, (topic_freq_bow_idxs, pred_topic_sent) in enumerate(zip(topics_freq_bow_idxs, pred_topic_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_topic_sent)
        
def print_summary(test_batch):
    feed_dict = get_feed_dict(test_batch)
    feed_dict[t_variables['batch_l']] = config.n_topic
    feed_dict[t_variables['keep_prob']] = 1.
    pred_topics_freq_bow_indices, pred_summary_token_idxs = sess.run([topics_freq_bow_indices, summary_beam_output_token_idxs], feed_dict=feed_dict)
    pred_summary_sents = idxs_to_sents(pred_summary_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Output sentences for each topic-----------')
    print('Item idx:', test_batch[0].item_idx)
    for i, (topic_freq_bow_idxs, pred_summary_sent) in enumerate(zip(topics_freq_bow_idxs, pred_summary_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_summary_sent)
        
    print('-----------Summaries-----------')
    for i, summary in enumerate(test_batch[0].summaries):
        print('SUMMARY %i :'%i, '\n', summary)

In [25]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','LM','','VALID:','TM','','','','LM','', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','NLL','KL','LOSS','PPL','NLL','KL','REG','NLL','KL', 'Beta']]))))

In [26]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, sent_loss_kl_categ_batch, sent_loss_kl_gmm_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, sent_loss_kl_categ, sent_loss_kl_gmm, topic_ppls], feed_dict = feed_dict)
   
        if sent_loss_kl_batch == np.inf:
            print('Nan occured')
            ckpt = tf.train.get_checkpoint_state(config.modeldir)
            model_checkpoint_path = ckpt.all_model_checkpoint_paths[-1]
            saver.restore(sess, model_checkpoint_path)            
            break
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, sent_loss_recon_train, sent_loss_kl_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, sent_loss_recon_dev, sent_loss_kl_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log, beta_eval = sess.run([tf.train.get_global_step(), beta])
            
            if loss_dev < loss_min:
                loss_min = loss_dev
                saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, '%.2f'%sent_loss_recon_train, '%.2f'%sent_loss_kl_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, '%.2f'%sent_loss_recon_dev, '%.2f'%sent_loss_kl_dev,  '%.3f'%beta_eval],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)

            print_summary(test_batches[1][1])
            print_sample(batch)
            
            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                        LM        VALID:  \
       Time  Ep    Ct    LOSS   PPL     NLL    KL   REG   NLL    KL    LOSS   
1        15   0     0  133.50  1036  122.98  0.43  0.97  9.12  1.15  126.60   
501      74   0   500  121.47   598  114.37  0.08  0.66  6.32  1.04  111.65   
1001     72   0  1000  120.07   576  113.30  0.13  0.52  6.05  0.82  111.37   
1501     73   0  1500  119.50   561  112.81  0.24  0.44  5.94  0.72  110.80   
2001     75   0  2000  119.19   549  112.51  0.36  0.38  5.86  0.65  110.46   
2276     56   1     0  119.09   545  112.41  0.41  0.35  5.82  0.62  110.30   
2776     74   1   500  118.79   537  112.12  0.50  0.31  5.74  0.57  109.84   
3276     72   1  1000  118.44   529  111.78  0.58  0.29  5.67  0.53  109.59   
3776     72   1  1500  118.35   523  111.70  0.65  0.26  5.61  0.50  109.43   
4276     72   1  2000  118.13   517  111.50  0.71  0.24  5.55  0.48  109.37   
4551     46   2     0  117.99   514  111.36  0.74  0.23  5.52  0.47  109.23   
5051     72   2   500  117.82   510  111.19  0.78  0.21  5.47  0.45  109.14   
5551     71   2  1000  117.67   505  111.05  0.83  0.20  5.43  0.43  108.98   
6051     61   2  1500  117.51   502  110.89  0.87  0.19  5.39  0.42  108.98   
6551     72   2  2000  117.42   498  110.81  0.90  0.18  5.35  0.41  108.74   
6826     33   3     0  117.38   497  110.76  0.92  0.17  5.33  0.41  108.88   
7326     62   3   500  117.31   494  110.70  0.96  0.16  5.30  0.40  108.78   
7826     72   3  1000  117.22   491  110.61  0.99  0.15  5.27  0.39  108.66   
8326     61   3  1500  117.14   489  110.53  1.02  0.15  5.24  0.39  108.67   
8826     73   3  2000  116.99   487  110.39  1.04  0.14  5.21  0.38  108.55   
9101     42   4     0  116.97   485  110.36  1.06  0.14  5.20  0.38  108.38   
9601     56   4   500  116.88   483  110.27  1.09  0.13  5.18  0.37  108.45   
10101    68   4  1000  116.82   481  110.21  1.11  0.13  5.15  0.37  108.37   
10601    69   4  1500  116.74   479  110.12  1.14  0.12  5.13  0.37  108.34   
11101    57   4  2000  116.69   477  110.08  1.16  0.12  5.11  0.37  108.45   
11376    47   5     0  116.66   476  110.04  1.17  0.11  5.10  0.37  107.93   
11876    56   5   500  116.61   475  110.00  1.20  0.11  5.08  0.37  107.96   
12376    68   5  1000  116.52   473  109.91  1.22  0.11  5.06  0.38  107.75   
12876    56   5  1500  116.45   471  109.85  1.25  0.10  5.04  0.38  107.77   
13376    56   5  2000  116.38   469  109.77  1.28  0.10  5.02  0.38  107.88   
...     ...  ..   ...     ...   ...     ...   ...   ...   ...   ...     ...   
100101   30  44     0  113.73   396  106.88  2.47  0.02  4.00  0.52  106.02   
100601   53  44   500  113.73   396  106.87  2.47  0.02  4.00  0.52  106.12   
101101   52  44  1000  113.73   396  106.87  2.47  0.02  4.00  0.52  106.05   
101601   52  44  1500  113.72   396  106.86  2.47  0.02  4.00  0.52  106.09   
102101   52  44  2000  113.72   396  106.86  2.48  0.02  3.99  0.52  106.09   
102376   29  45     0  113.71   396  106.85  2.48  0.02  3.99  0.52  105.45   
102876   53  45   500  113.70   396  106.85  2.48  0.02  3.99  0.52  105.46   
103376   52  45  1000  113.70   396  106.84  2.48  0.02  3.98  0.52  105.68   
103876   52  45  1500  113.69   395  106.84  2.48  0.02  3.98  0.52  105.68   
104376   63  45  2000  113.68   395  106.83  2.49  0.02  3.98  0.52  105.41   
104651   29  46     0  113.68   395  106.83  2.49  0.02  3.98  0.52  105.70   
105151   53  46   500  113.68   395  106.83  2.49  0.02  3.97  0.52  105.49   
105651   52  46  1000  113.67   395  106.82  2.49  0.02  3.97  0.52  105.72   
106151   52  46  1500  113.67   395  106.82  2.49  0.02  3.97  0.52  105.78   
106651   53  46  2000  113.66   395  106.81  2.50  0.02  3.97  0.52  105.78   
106926   30  47     0  113.65   395  106.81  2.50  0.02  3.96  0.52  105.83   
107426   54  47   500  113.65   395  106.80  2.50  0.02  3.96  0.52  105.98   
107926   54  47  1000  113.65   394  106.80  2.50  0.02  3.96  0.52  106.

-----------Output sentences for each topic-----------
Item idx: B000VB7EFW
0  BOW: zipper open strap zippers flap top front side handle main
0  SENTENCE: the main flap on the front flap clips on the front flap clips on the bottom flap clips on the bottom flap
1  BOW: cover keyboard color apple easy protector love hard screen put
1  SENTENCE: the keyboard keyboard cover fits perfectly and the keyboard cover is a little difficult to type on
2  BOW: months broke started year years bought weeks month ago week
2  SENTENCE: the first one lasted about # months ago and the top piece broke within the first week
3  BOW: inch price perfect size made small bought carry hp big
3  SENTENCE: my perfect fit for my ipad perfectly ! !
4  BOW: listed test sewn fair refund returning late stated website manufacturer
4  SENTENCE: it is a little bit bigger than i thought it would be , but it 's not a problem
5  BOW: smell strong chemical smells days odor bad opened package packaging
5  SENTENCE: one of the o

# confirm variables

In [27]:
_prob_topic, _prob_topic_sents, _prob_topic_infer, _means_topic_infer = debug_value([prob_topic, prob_topic_sents, prob_topic_infer, means_topic_infer], return_value=True)

In [28]:
batch_i = 4
_prob_topic_sents[batch_i], _prob_topic_infer[batch_i]

(array([0.01681865, 0.00085679, 0.05174972, 0.09216808, 0.00016615,
        0.00087459, 0.00118037, 0.02513456, 0.01622178, 0.14665115,
        0.0011753 , 0.02991384, 0.03108656, 0.00118961, 0.11844516,
        0.01804251, 0.01136509, 0.12562811, 0.05762221, 0.01317067,
        0.00810287, 0.06980555, 0.00149514, 0.00750153, 0.00170939,
        0.01077554, 0.13081829, 0.00062634, 0.00238444, 0.00731998],
       dtype=float32),
 array([9.0836927e-02, 1.0987123e-03, 1.5486477e-01, 3.6601584e-02,
        8.1031001e-05, 3.8817880e-04, 1.8645950e-02, 6.4254373e-02,
        1.1557489e-02, 1.8391356e-01, 1.7670565e-03, 9.3876962e-03,
        3.3649437e-02, 2.5801428e-03, 1.2207399e-01, 2.7256794e-03,
        5.1550437e-03, 5.9668675e-02, 1.9978454e-02, 3.9733057e-03,
        7.7147051e-03, 6.4633414e-02, 3.8139557e-03, 1.5572711e-02,
        3.4457527e-03, 1.8684692e-03, 5.4347310e-02, 7.9502229e-04,
        2.2245880e-02, 2.3606885e-03], dtype=float32))

In [29]:
_means_topic_infer[0][:, :4]

array([[-2.56992847e-01,  6.93834186e-01, -8.92492712e-01,
         2.42564678e-01],
       [-4.57342863e-01, -4.06658985e-02,  1.23339975e+00,
        -5.94517171e-01],
       [-4.11060035e-01,  5.96444011e-02, -9.75041866e-01,
         1.38034391e+00],
       [ 5.46178877e-01, -2.98609585e-01,  8.14010501e-01,
         9.63758230e-01],
       [-4.58527356e-02,  7.22829029e-02, -1.19237393e-01,
        -3.92585009e-01],
       [-2.96975613e-01,  1.55666143e-01, -2.59647340e-01,
         3.35099936e-01],
       [-3.07893324e+00,  1.74073529e+00,  3.32384348e-01,
        -1.66081464e+00],
       [-3.01134288e-01,  4.63835001e-01, -1.16023615e-01,
        -1.26744211e-02],
       [ 2.22409916e+00, -2.20579481e+00, -4.48575258e+00,
        -2.86763134e+01],
       [-8.55066776e-02,  2.23345548e-01, -1.50715339e+00,
         2.03623366e+00],
       [-1.57129437e-01,  3.82197917e-01,  6.79618537e-01,
        -7.08601534e-01],
       [ 1.61208048e-01,  6.48200572e-01, -8.06558877e-03,
      

In [30]:
w_means_topic, b_means_topic = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "topic/dec/mean_topic")

pred_topic_embeddings, pred_topic_bow, pred_means_topic, pred_logvars_topic, pred_token_idxs, _w_means_topic, _b_means_topic, _w_mean_topic_infer = \
                                sess.run([topic_embeddings, topic_bow, means_topic, logvars_topic, topic_beam_output_token_idxs, w_means_topic, b_means_topic, w_mean_topic_infer], 
                                         feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})

pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)

pred_topics_freq_bow_indices = np.argsort(pred_topic_bow, 1)[:, ::-1][:, :10]
pred_topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]

In [31]:
for idxs in pred_topics_freq_bow_idxs:
    print([idx_to_word[idx] for idx in idxs])

['zipper', 'strap', 'flap', 'open', 'zippers', 'velcro', 'main', 'metal', 'side', 'front']
['cover', 'keyboard', 'apple', 'color', 'easy', 'protector', 'love', 'screen', 'hard', 'put']
['months', 'broke', 'year', 'years', 'started', 'bought', 'month', 'weeks', 'week', 'ago']
['perfect', 'inch', 'price', 'size', 'carry', 'small', 'bought', 'carrying', 'made', 'big']
['listed', 'test', 'sewn', 'fair', 'refund', 'late', 'stated', 'returning', 'website', 'manufacturer']
['smell', 'days', 'strong', 'bad', 'smells', 'chemical', 'odor', 'package', 'packaging', 'opened']
[';', '&', 'pro', 'air', 'smell', 'description', 'laptops', 'ordered', 'big', 'tablet']
[':', 'air', 'bit', 'cheap', '-', 'pretty', 'fine', 'close', 'thin', 'protect']
['!', 'awesome', 'loves', 'daughter', 'absolutely', 'amazing', 'loved', 'christmas', 'gift', 'wonderful']
['straps', 'back', 'handle', 'shoulder', 'pack', 'strap', 'heavy', 'bags', "'ve", 'zippers']
['scratches', 'retina', 'shell', 'hard', 'mbp', 'weight', 'prot

In [32]:
pred_topic_embeddings[:, :10]

array([[ 0.04221381,  0.07032954,  0.02605449, -0.14514676, -0.10837159,
        -0.13675061,  0.04316118,  0.21489286,  0.02900301, -0.11756933],
       [-0.00191224,  0.09261359,  0.01232868, -0.0661851 ,  0.07643441,
         0.12696671, -0.20366614,  0.1984897 ,  0.17354025,  0.02899385],
       [ 0.02067169, -0.20038156, -0.09364118,  0.21172738, -0.02731904,
        -0.04265526,  0.05918429,  0.1413685 , -0.12565222,  0.03097908],
       [ 0.06493654,  0.0132221 ,  0.01086067,  0.04034178,  0.4537829 ,
         0.11259412,  0.00850064,  0.14192611, -0.03143219, -0.27809662],
       [-0.29869643,  0.26431245, -0.24967694, -0.13403602, -0.17315197,
         0.24033956,  0.28124785,  0.10158894,  0.15352075, -0.12799466],
       [-0.5816298 , -0.43490657,  0.17611949,  0.18747905,  0.13666977,
        -0.30238554, -0.34233955,  0.47408575,  0.08233652, -0.03812001],
       [-0.01370848,  0.33514827,  0.0198018 , -0.27161032,  0.32002565,
        -0.52758247,  0.13847867,  0.06868012

In [33]:
pred_topic_bow

array([[1.9581764e-06, 6.6142122e-05, 1.2562939e-04, ..., 1.0677581e-04,
        2.9342832e-02, 9.0854871e-04],
       [2.6542670e-03, 7.8307443e-05, 1.5476791e-04, ..., 1.0871761e-05,
        4.9786659e-06, 2.2817678e-06],
       [3.2476793e-05, 1.3679019e-03, 2.5892191e-04, ..., 3.5754842e-06,
        1.2249883e-03, 1.1139793e-05],
       ...,
       [2.4093279e-07, 2.0777383e-04, 1.8917325e-04, ..., 1.6858297e-05,
        3.9735093e-04, 3.7389019e-04],
       [5.4072992e-05, 3.4708713e-04, 9.8067081e-05, ..., 1.0476207e-05,
        1.7639939e-05, 6.9869757e-06],
       [2.8947432e-02, 1.1348367e-05, 8.0334677e-05, ..., 1.4319714e-06,
        6.2947379e-07, 1.1913493e-06]], dtype=float32)

In [34]:
_w_means_topic

array([[-0.09132125,  0.06384595,  0.05307246, ...,  0.11143875,
         0.0583626 , -0.00763746],
       [-0.08056675,  0.05794175,  0.1096086 , ...,  0.02227432,
        -0.11894352, -0.00020836],
       [-0.01657696, -0.03343681,  0.13245685, ..., -0.05597756,
        -0.07617768, -0.19336337],
       ...,
       [ 0.07739904, -0.05467948, -0.03028192, ...,  0.10324294,
         0.16739792,  0.08804034],
       [-0.06471275,  0.05133058, -0.05663415, ..., -0.03235153,
         0.05406563, -0.08967461],
       [ 0.03386614, -0.02483214, -0.02778944, ...,  0.0683267 ,
         0.02738107, -0.07660544]], dtype=float32)

In [35]:
_b_means_topic

array([ 0.00559006,  0.05444232, -0.04536091,  0.18858178,  0.05161314,
        0.09153585, -0.08774164,  0.2157619 ,  0.00775018, -0.11418495,
        0.02302523,  0.05408015, -0.01412298,  0.25445768, -0.07876244,
        0.09265354, -0.03667201,  0.13205457,  0.04882353,  0.1650994 ,
       -0.11794744,  0.02307192, -0.12431027,  0.14413537, -0.14899743,
       -0.10817293,  0.14402367, -0.02406884,  0.17764609, -0.04606541,
        0.07183758,  0.00588563], dtype=float32)

In [36]:
pred_means_topic

array([[-2.57760793e-01,  6.92940652e-01, -8.86387587e-01,
         2.32742757e-01,  9.04477775e-01, -3.78198564e-01,
         6.45827174e-01, -4.35379326e-01,  1.85461867e+00,
         2.75790304e-01,  1.85874355e+00, -1.98776901e+00,
         1.86697483e+00,  2.70285726e+00,  7.95651555e-01,
         1.16843891e+00, -2.92420149e-01,  4.81955886e-01,
         2.64107585e+00, -1.38735437e+00,  3.86276960e-01,
         2.49031812e-01, -2.64650798e+00,  5.41839659e-01,
        -7.88616300e-01, -1.70691192e+00,  1.99204385e+00,
         2.27924943e+00,  2.09087896e+00, -1.38838899e+00,
         9.71355259e-01,  8.49284589e-01],
       [-4.51019406e-01, -4.28144448e-02,  1.23017800e+00,
        -5.81427395e-01, -1.11459160e+00, -8.72108400e-01,
         9.18848634e-01,  3.49255800e-01,  3.26688600e+00,
         2.31632733e+00,  2.68863630e+00,  1.14830577e+00,
        -1.39267373e+00, -1.25154883e-01, -2.60801911e-01,
         8.40898752e-01,  9.82761979e-01, -1.31821489e+00,
        -6.96

In [37]:
_w_mean_topic_infer[:, :10, 0]

array([[ 1.46229323e-02,  1.00070061e-02, -1.43917352e-02,
         2.56802281e-03, -3.66200729e-05,  3.88381211e-03,
        -9.58103314e-03, -3.79253388e-03, -3.20388447e-03,
        -9.68570262e-03],
       [-5.87334018e-03, -7.74201099e-03, -1.63234342e-02,
        -1.80622004e-03, -1.69612712e-03, -3.15694674e-03,
        -2.67671468e-03, -9.32745636e-03,  4.14251350e-03,
         5.37557947e-03],
       [ 7.78287929e-03, -8.10541306e-03, -9.05863661e-03,
        -3.13145341e-03,  6.40133582e-03, -9.14698525e-04,
        -3.00810719e-03,  2.94189272e-03, -3.06034321e-03,
         1.01561295e-02],
       [-8.63092020e-03, -9.98362713e-03,  2.42341636e-03,
         6.84057176e-03,  9.59877204e-03,  5.46535384e-03,
         1.05908290e-02,  1.03722513e-02,  2.74874712e-03,
         5.13785228e-04],
       [-6.19821600e-04,  4.41455981e-03,  4.71293414e-03,
        -9.49300826e-03, -4.08250466e-03,  1.10118845e-02,
        -3.50344693e-03, -2.48622731e-03,  1.30777154e-02,
         7.

In [38]:
_b_means_topic

array([ 0.00559006,  0.05444232, -0.04536091,  0.18858178,  0.05161314,
        0.09153585, -0.08774164,  0.2157619 ,  0.00775018, -0.11418495,
        0.02302523,  0.05408015, -0.01412298,  0.25445768, -0.07876244,
        0.09265354, -0.03667201,  0.13205457,  0.04882353,  0.1650994 ,
       -0.11794744,  0.02307192, -0.12431027,  0.14413537, -0.14899743,
       -0.10817293,  0.14402367, -0.02406884,  0.17764609, -0.04606541,
        0.07183758,  0.00588563], dtype=float32)

In [39]:
_enc_state_infer, _means_topic_infer = debug_value([enc_state_infer, means_topic_infer], return_value=True)

NameError: name 'enc_state_infer' is not defined

In [ ]:
_enc_state_infer.shape

In [ ]:
_means_topic_infer[0]